# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [10]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pkg_resources

import sklearn  

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.script_run_config import ScriptRunConfig
from azureml.core import Environment

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import shutil


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.26.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

experiment_name = 'fraud_detection_hypervisor'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()
print(experiment)

Workspace name: quick-starts-ws-142871
Azure region: southcentralus
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-142871
Experiment(Name: fraud_detection_hypervisor,
Workspace: quick-starts-ws-142871)


In [5]:
dataset = Dataset.get_by_name(workspace=ws, name='Banking_Transactions')
transactions_df = dataset.to_pandas_dataframe()
transactions_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [2]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = 'cdbe0b43-92a0-4715-838a-f2648cc7ad21'
resource_group = 'aml-quickstarts-142814'
workspace_name = 'quick-starts-ws-142814'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace=ws, name='Banking_Transactions')
transactions_df = dataset.to_pandas_dataframe()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RKYGT3WZ4 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


UserErrorException: UserErrorException:
	Message: You are currently logged-in to 660b3398-b80e-49d2-bc5b-ac1dc93b5254 tenant. You don't have access to cdbe0b43-92a0-4715-838a-f2648cc7ad21 subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = 
 [SubscriptionInfo(subscription_name='Udacity CloudLabs Sub - 01', subscription_id='510b94ba-e453-4417-988b-fbdc37b55ca7')]. 
 Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "You are currently logged-in to 660b3398-b80e-49d2-bc5b-ac1dc93b5254 tenant. You don't have access to cdbe0b43-92a0-4715-838a-f2648cc7ad21 subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = \n [SubscriptionInfo(subscription_name='Udacity CloudLabs Sub - 01', subscription_id='510b94ba-e453-4417-988

In [6]:
# Calculate the fraction of data points that are fraudulent
def fraudulent_percentage(transaction_df):
    '''Calculate the fraction of all data points that have a 'Class' label of 1; fraudulent.
       :param transaction_df: Dataframe of all transaction data points; has a column 'Class'
       :return: A fractional percentage of fraudulent data points/all points
    '''
    # counts for all classes
    counts = transactions_df['Class'].value_counts()
    
    # get fraudulent and valid cnts
    fraud_cnts = counts[1]
    valid_cnts = counts[0]
    
    # calculate percentage of fraudulent data
    fraud_percentage = fraud_cnts/(fraud_cnts+valid_cnts)
    
    return fraud_percentage

In [7]:
# call the function to calculate the fraud percentage
fraud_percentage = fraudulent_percentage(dataset)

print('Fraudulent percentage = ', fraud_percentage)
print('Total # of fraudulent pts: ', fraud_percentage*transactions_df.shape[0])
print('Out of (total) pts: ', transactions_df.shape[0])

Fraudulent percentage =  0.001727485630620034
Total # of fraudulent pts:  492.0
Out of (total) pts:  284807


In [8]:
# create the features dataframe
x = transactions_df.iloc[:, :-1]
# create the data label
y = transactions_df['Class']

# split data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=50, shuffle=True)


In [9]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# Create compute cluster
# Choose a name for the computer cluster:
amlcompute_cluster_name = "compute-cluster"

# Check if the cluster already exists:
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('The cluster already exists')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [27]:
from azureml.data.dataset_factory import TabularDatasetFactory

url_2 = 'https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c534768_creditcardfraud/creditcardfraud.zip'

ds = TabularDatasetFactory.from_delimited_files(url)

DatasetValidationError: DatasetValidationError:
	Message: Cannot load any data from the specified path. Make sure the path is accessible and contains data.
ScriptExecutionException was caused by StreamAccessException.
  StreamAccessException was caused by ValidationException.
    Unable to read file using Unicode (UTF-8). Attempted read range 0:20971520. Lines read in the range 0. Decoding error: [REDACTED]
      Failed due to inner exception of type: DecoderFallbackException
| session_id=69b95312-5563-49cd-8ba0-1dea9e914d43
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Cannot load any data from the specified path. Make sure the path is accessible and contains data.\nScriptExecutionException was caused by StreamAccessException.\n  StreamAccessException was caused by ValidationException.\n    Unable to read file using Unicode (UTF-8). Attempted read range 0:20971520. Lines read in the range 0. Decoding error: [REDACTED]\n      Fai

In [28]:
from sklearn.ensemble import RandomForestClassifier
# Create an Azure environment
curated_env = Environment.get(workspace=ws, name='AzureML-Tutorial')
# Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

# Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--n_estimators": choice(1, 2, 4, 8, 16, 32, 64, 100, 150, 200),
    "--max_depth": choice(1, 2, 4, 8, 16, 32),
    "--min_samples_split": choice(10, 20, 30, 40, 50, 60, 70, 80, 90, 100),
    "--max_features": choice(1, 5, 10 , 15, 20, 25, 30)
})

#TODO: Create your estimator and hyperdrive config
if "training" not in os.listdir():
    os.mkdir("./training")

script_folder="./training"

# Create a copy the training script into the script_folder
shutil.copy("./train.py", script_folder)
# Create a ScriptRunConfig estimator for use with train.py
est = ScriptRunConfig(source_directory="./training",
        script="train.py",
        compute_target=aml_compute,
        environment=curated_env
)

hyperdrive_run_config = HyperDriveConfig(run_config=est,
                                     hyperparameter_sampling=param_sampling,
                                     policy=early_termination_policy,
                                     primary_metric_name='AUC_weighted',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [29]:
# Submit the experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [30]:
# Show run details with the widget
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [31]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_a880cdc6-dc11-45bb-8912-0af54eda368c
Web View: https://ml.azure.com/runs/HD_a880cdc6-dc11-45bb-8912-0af54eda368c?wsid=/subscriptions/510b94ba-e453-4417-988b-fbdc37b55ca7/resourcegroups/aml-quickstarts-142871/workspaces/quick-starts-ws-142871&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-04-16T20:53:42.701918][API][INFO]Experiment created<END>\n""<START>[2021-04-16T20:53:43.299273][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-04-16T20:53:43.592029][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-04-16T20:53:45.0581843Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_a880cdc6-dc11-45bb-8912-0af54eda368c
Web View: https://ml.azure.com/runs/HD_a880cdc6-dc11-45bb-8912-0af54eda368c?wsid=/subscriptions/510b94ba-e453-4417-9

{'runId': 'HD_a880cdc6-dc11-45bb-8912-0af54eda368c',
 'target': 'compute-cluster',
 'status': 'Canceled',
 'startTimeUtc': '2021-04-16T20:53:42.45451Z',
 'endTimeUtc': '2021-04-16T21:02:49.306977Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name": 

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [28]:
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

AttributeError: 'NoneType' object has no attribute 'get_details'

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service